# After refactoring...

In [ ]:
import signalflow as sf
from pixasonics.core import App, Mapper
from pixasonics.features import MeanPixelValue
from pixasonics.synths import Theremin

# Create app
app = App()

In [2]:
# load an image
img_path = "images/cellular_dataset/Timepoint_001_220518-ST_C03_s1.jpg"
img = app.load_image(img_path)

In [3]:
app.probe_height = 100
app.probe_width = 10
app.probe_x = 100
app.probe_y = 100

In [ ]:
# Create objects
mean_pix = MeanPixelValue()
theremin = Theremin()
pix2freq = Mapper(mean_pix, theremin["frequency"], exponent=2, out_high=1000)

# Add objects to app
app.attach_feature(mean_pix)
app.attach_synth(theremin)
app.attach_mapper(pix2freq)


In [ ]:
pix2freq.ui

In [ ]:
# test adding another mapping: mean pixel value to amplitude with an exponential curve
pix2amp = Mapper(mean_pix, theremin["amplitude"], exponent=2)
app.attach_mapper(pix2amp)
pix2amp.ui

In [4]:
pix2pan = Mapper(mean_pix, theremin["panning"])
app.attach_mapper(pix2pan)

In [ ]:
app.mappers

In [ ]:
# remove a mapper
app.detach_mapper(pix2amp)
app.mappers

In [ ]:
# remove old frequency mapping and add a new one with different scaling
app.detach_mapper(pix2freq)
pix2freq = Mapper(mean_pix, theremin["frequency"], exponent=2, out_low=100, out_high=2000)
app.attach_mapper(pix2freq)
app.mappers

In [ ]:
graph = sf.AudioGraph.get_shared_graph()
print(graph.structure)
print(graph.status)

# Import widget stuff

In [1]:
from ipywidgets import ToggleButton, Button, ButtonStyle, Layout, jslink, IntText, IntSlider, FloatSlider, GridBox, GridspecLayout, Box, VBox, Label, Accordion

# Mapper card proto

In [ ]:
mapper_label = Label(value="Mapper", style=dict(
    font_weight='bold',
    font_size='2em'))
mapper_id = Label(value="# ID", style=dict(
    font_weight='bold',
    font_size='1em',
    text_color='gray'))

top_row = Box([mapper_label, mapper_id], layout=Layout(justify_content='space-between'))

from_label = Label(value="From:")
from_value = Label(value="MeanPixelValue")
from_row = Box([from_label, from_value], layout=Layout(justify_content='space-between'))

to_label = Label(value="To:")
to_value = Label(value="Theremin Frequency")
to_row = Box([to_label, to_value], layout=Layout(justify_content='space-between'))

detach_btn = Button(description="Detach", button_style='danger', icon='unlink')
detach_btn.on_click(lambda x: print("Detaching mapper"))
detach_row = Box([detach_btn], layout=Layout(justify_content='flex-end'))

mapper_card = GridBox(children=[top_row, from_row, to_row, detach_row],
                        layout=Layout(
                            width='auto', 
                            grid_template_columns='auto', 
                            grid_template_rows='2fr 1.5fr 1.5fr 1fr',
                            max_width='300px',
                            min_height='200px',
                            border='2px solid black',
                            padding='5px',
                            margin='5px'))

mapper_card


# Carousel proto

In [ ]:
# Create a vertical carousel of mapper cards
mapper_cards = [mapper_card for i in range(10)]
mapper_carousel = Box(children=mapper_cards, layout=Layout(overflow='scroll',
                                                            border='3px solid black',
                                                            padding='5px',
                                                            width='auto',
                                                            max_width='300px',
                                                            max_height='500px',
                                                            flex_flow='column',
                                                            display='flex'))
mapper_carousel

In [ ]:
# access the children of the carousel
mapper_carousel.children

# remove a card
mapper_carousel.children = mapper_carousel.children[:-1]

mapper_carousel

# App layout proto

In [ ]:
from ipywidgets import ToggleButton, Button, ButtonStyle, Layout, jslink, IntText, IntSlider, FloatSlider, GridBox, GridspecLayout, Box, VBox, Label, Accordion
from pixasonics.ui import AppUI, ProbeSettings, AudioSettings, Model, find_widget_by_tag

probe_settings = ProbeSettings()
audio_settings = AudioSettings()

app_ui = AppUI(
    probe_settings,
    audio_settings
    )

app_ui()

In [2]:
# add a new canvas
test_canvas = Button(description="Test Canvas", layout=Layout(width='500px', height='500px'))
app_ui.box.children[0].children = [test_canvas]

In [ ]:
# add a new canvas the new way
test_canvas = Button(description="Test Canvas", layout=Layout(width='500px', height='500px'))
app_canvas = find_widget_by_tag(app_ui.box, "app_canvas")
app_canvas.children = [test_canvas]

In [5]:
# add buttons to the features carousel
features_carousel = app_ui.box.children[1].children[0].children[2]
features_carousel.children = list(features_carousel.children) + [Button(description=f"Feature {i}") for i in range(10)]

In [6]:
# add buttons the new way
test_buttons = [Button(description=f"Feature {i}") for i in range(10)]
features_carousel = find_widget_by_tag(app_ui.box, "features_carousel")
features_carousel.children = list(features_carousel.children) + test_buttons

In [5]:
# delete all buttons from the features carousel
features_carousel.children = []